## Introduction

This tutorial will introduce the [Valve's Dota 2 API](https://wiki.teamfortress.com/wiki/WebAPI#Dota_2) and several methods to anaylze a Dota 2 match and a Dota 2 player.

[Dota 2](http://www.dota2.com/play/) is a multiplayer online battle arena ([MOBA](https://en.wikipedia.org/wiki/Multiplayer_online_battle_arena)) video game developed and published by Valve Corporation. Dota 2 is played in matches between two teams of five players, with each team occupying and defending their own separate base on the map. Each of the ten players independently controls a powerful character, known as a "hero", who all have unique abilities and differing styles of play.

According to Valve, there are more than 1,000,000 active Dota 2 players in the world and huge amount of mathes being played. By analyzing the Dota 2 match data, we could ...

### Tutorial content
In this tutorial, we will show how to collect data of:
- Hero information
- Single Dota2 match
- Match history of a player

Also, we will cover following analyzing method:
- [Analyze a match: Who is MVP?](#Analyze-a-match:-Who-is-MVP?)
- [Analyze a player: Top records](#Analyze-a-player:-Top-records)
- [Analyze a player: Radar chart](#Analyze-a-player:-Radar-chart)

## Work with Dota 2 APIs

In [38]:
import json
import requests
import numpy as np
import time, datetime

In [2]:
def read_api_key(filepath):
    with open('api_key.txt', 'r') as f:
        return f.read().replace('\n','')
def get_heroes(api_key):
    portrait_url_base = 'http://cdn.dota2.com/apps/dota2/images/heroes/'
    url = 'http://api.steampowered.com/IEconDOTA2_570/GetHeroes/v1'
    params = {"key" : api_key}
    response = requests.get(url, params=params)
    data = json.loads(response.text)['result']
    idx = 0
    heroid2idx = {}
    for hero in data['heroes']:
        heroid2idx[hero['id']] = idx
        hero['localized_name'] = hero['name'].replace('npc_dota_hero_', '').replace('_', ' ').title()
        hero['portrait_url'] = portrait_url_base + hero['name'].replace('npc_dota_hero_', '') + '_sb.png'
        idx += 1
    return (data['heroes'], heroid2idx)
def get_match_details(match_id, api_key):
    url = 'http://api.steampowered.com/IDOTA2Match_570/GetMatchDetails/v001/'
    params = {"key" : api_key, "match_id" : match_id}
    response = requests.get(url, params=params)
    data = json.loads(response.text)['result']
    return data
def get_match_history(account_id, api_key, game_mode = 1, matches_requested = 25, start_at_match_id = None):
    url = 'http://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/v001/'
    params = {"key" : api_key, \
              "account_id" : account_id, \
              "game_mode" : game_mode, \
              "matches_requested" : matches_requested}
    if (start_at_match_id != None):
        params['start_at_match_id'] = start_at_match_id
    response = requests.get(url, params=params)
    data = json.loads(response.text)['result']
    return data

base_url = 'http://api.steampowered.com/IDOTA2Match_570/GetMatchDetails/v001/'
api_key = read_api_key('api_key.txt')
match_id = '3777928249'
my_account_id = '141393206'
heroes, heroid2idx = get_heroes(api_key)
#print(get_match_details(match_id, api_key))

In [3]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
def get_png_md(png_url):
    return '![](%s?raw=true)'%png_url

In [43]:
def process_match(data):
    cnt = 0
    total_hero_damage = [0, 0]
    total_kill = [0, 0]
    total_gold = [0, 0]
    total_tower_damage = [0, 0]
    for player in data['players']:
        cnt += 1
        player['overall_gold'] = player['gold']+player['gold_spent']
        if (cnt <= 5):
            player['camp'] = 0
            total_hero_damage[0] += player['hero_damage']
            total_kill[0] += player['kills']
            total_gold[0] += player['overall_gold']
        else:
            player['camp'] = 1
            total_hero_damage[1] += player['hero_damage']
            total_kill[1] += player['kills']
            total_gold[1] += player['overall_gold']
    for player in data['players']:
        camp = player['camp']
        player['KDA'] = (player['kills']+player['assists'])/(player['deaths']+1.0)
        if (total_hero_damage[camp] == 0):
            player['hero_damage_per'] = 0.0
        else:
            player['hero_damage_per'] = player['hero_damage']/(total_hero_damage[camp]*1.0)*100.0
        if (total_kill[camp] == 0):
            player['in_battle_per'] = 0.0
        else:
            player['in_battle_per'] = (player['kills']+player['assists'])/(total_kill[camp]*1.0)*100.0
        player['gold_per'] = player['overall_gold']/(total_gold[camp]*1.0)*100.0
    return data
    
def show_match(data):
    cols = 'Players|Heroes|KDA|Last Hits/Denies|Tower Damage|Hero Healing|In Battle %|Damage %|Gold %|MVP|\n'\
           ':-|:-|:-|:-|:-|:-|:-|:-|:-|:-|\n'
    fmat = '%s|%s**lvl %d**|**%.1f**(%d/%d/%d)|%d/%d|%d|%d|%.1f%%|%.1f%%|%.1f%%|%.1f|\n'
    if (data['radiant_win']):
        radiant_res = ' (Victory)'
        dire_res = ''
    else:
        radiant_res = ''
        dire_res = ' (Victory)'
    md = '## Match Detail\n\n'
    md += '- MatchID: %s\n'%data['match_id']
    md += '- Start Time: %s\n'%\
            (datetime.datetime.fromtimestamp(int(data['start_time'])).strftime('%Y-%m-%d %H:%M:%S'))
    md += '- Duration: %d min\n'%(int(data['duration'])//60)
    md += '***\n'
    md += '### Radient%s\n\n'%radiant_res
    md += cols
    cnt = 0
    for player in data['players']:
        md += fmat%(player['account_id'], \
                    get_png_md(heroes[heroid2idx[player['hero_id']]]['portrait_url']), \
                    player['level'], \
                    player['KDA'], \
                    player['kills'], \
                    player['deaths'], \
                    player['assists'], \
                    player['last_hits'], \
                    player['denies'], \
                    player['tower_damage'], \
                    player['hero_healing'], \
                    player['in_battle_per'], \
                    player['hero_damage_per'], \
                    player['gold_per'], \
                    player['hero_damage_per']/player['gold_per'])
        cnt += 1
        if (cnt == 5):
            md += '***\n### Dire%s\n\n'%dire_res
            md += cols
    printmd(md)
#print(process_match(get_match_details(3777836789, api_key)))
show_match(process_match(get_match_details(3777836789, api_key)))

## Match Detail

- MatchID: 3777836789
- Start Time: 2018-03-12 15:21:34
- Duration: 54 min
***
### Radient

Players|Heroes|KDA|Last Hits/Denies|Tower Damage|Hero Healing|In Battle %|Damage %|Gold %|MVP|
:-|:-|:-|:-|:-|:-|:-|:-|:-|:-|
136383633|![](http://cdn.dota2.com/apps/dota2/images/heroes/phantom_assassin_sb.png?raw=true)**lvl 25**|**5.0**(29/8/16)|520/18|4803|0|83.3%|37.6%|38.1%|1.0|
141393206|![](http://cdn.dota2.com/apps/dota2/images/heroes/bristleback_sb.png?raw=true)**lvl 25**|**1.9**(11/13/16)|268/12|396|0|50.0%|18.3%|21.7%|0.8|
142099634|![](http://cdn.dota2.com/apps/dota2/images/heroes/vengefulspirit_sb.png?raw=true)**lvl 24**|**2.3**(2/11/26)|53/7|188|0|51.9%|8.7%|12.0%|0.7|
134202447|![](http://cdn.dota2.com/apps/dota2/images/heroes/zuus_sb.png?raw=true)**lvl 25**|**3.4**(9/10/28)|174/4|250|80|68.5%|30.5%|13.4%|2.3|
136312013|![](http://cdn.dota2.com/apps/dota2/images/heroes/legion_commander_sb.png?raw=true)**lvl 23**|**0.9**(3/13/10)|249/9|569|924|24.1%|4.9%|14.8%|0.3|
***
### Dire (Victory)

Players|Heroes|KDA|Last Hits/Denies|Tower Damage|Hero Healing|In Battle %|Damage %|Gold %|MVP|
:-|:-|:-|:-|:-|:-|:-|:-|:-|:-|
58519674|![](http://cdn.dota2.com/apps/dota2/images/heroes/juggernaut_sb.png?raw=true)**lvl 25**|**5.2**(24/7/18)|374/12|8069|22728|77.8%|37.7%|32.8%|1.1|
160999411|![](http://cdn.dota2.com/apps/dota2/images/heroes/dark_willow_sb.png?raw=true)**lvl 25**|**3.2**(11/12/30)|49/2|1319|0|75.9%|17.0%|16.5%|1.0|
119448368|![](http://cdn.dota2.com/apps/dota2/images/heroes/pudge_sb.png?raw=true)**lvl 22**|**2.5**(4/10/24)|23/0|838|1682|51.9%|10.7%|12.8%|0.8|
89702175|![](http://cdn.dota2.com/apps/dota2/images/heroes/undying_sb.png?raw=true)**lvl 25**|**1.9**(5/18/31)|96/1|808|1282|66.7%|12.1%|12.4%|1.0|
205436003|![](http://cdn.dota2.com/apps/dota2/images/heroes/obsidian_destroyer_sb.png?raw=true)**lvl 25**|**4.4**(10/7/25)|343/18|3438|0|64.8%|22.5%|25.6%|0.9|


## Visualize a match

## Analyze a match: Who is MVP?

## Analyze a player: Top records

In [5]:
def get_all_match_history(account_id, api_key, game_mode):
    matches = []
    last_match = None
    results_remaining = 100
    while results_remaining != 0:
        time.sleep(1)
        if (last_match == None):
            data = get_match_history(account_id, api_key, game_mode, matches_requested = 100)
        else:
            data = get_match_history(account_id, api_key, game_mode, \
                                     matches_requested = 100, \
                                     start_at_match_id = last_match)
        results_remaining = data['results_remaining']
        if (matches != []):
            matches = matches[:-1]+data['matches']
        else:
            matches = data['matches']
        last_match = matches[-1]['match_id']
    return matches
my_hist = get_all_match_history(my_account_id, api_key, game_mode = 1)

In [6]:
def fav_heroes(hist, account_id):
    counter = {}
    for hero in heroes:
        #cnt, win, kill, death, assist, kda
        counter[hero['id']] = {'cnt' : 0, \
                               'win' : 0, \
                               'kills' : 0, \
                               'deaths' : 0, \
                               'assists' : 0, \
                               'KDA' : 0.0, \
                               'GPM' : 0, \
                               'XPM' : 0}
    cnt = 0
    for match in hist:
        cnt += 1
        print("%d/500"%cnt)
        match_detail = process_match(get_match_details(match['match_id'], api_key))
        for player in match_detail['players']:
            if (str(player['account_id']) == str(account_id)):
                counter[player['hero_id']]['cnt'] += 1
                counter[player['hero_id']]['win'] += player['camp'] ^ match_detail['radiant_win']
                counter[player['hero_id']]['kills'] += player['kills']
                counter[player['hero_id']]['deaths'] += player['deaths']
                counter[player['hero_id']]['assists'] += player['assists']
                counter[player['hero_id']]['KDA'] += player['KDA']
                counter[player['hero_id']]['GPM'] += player['gold_per_min']
                counter[player['hero_id']]['XPM'] += player['xp_per_min']
    for hero in heroes:
        if (counter[hero['id']]['cnt'] != 0):
            print("%s:%d"%(hero['localized_name'], counter[hero['id']]['cnt']))
    return counter
counter = fav_heroes(my_hist, my_account_id)

1/500
2/500
3/500
4/500
5/500
6/500
7/500
8/500
9/500
10/500
11/500
12/500
13/500
14/500
15/500
16/500
17/500
18/500
19/500
20/500
21/500
22/500
23/500
24/500
25/500
26/500
27/500
28/500
29/500
30/500
31/500
32/500
33/500
34/500
35/500
36/500
37/500
38/500
39/500
40/500
41/500
42/500
43/500
44/500
45/500
46/500
47/500
48/500
49/500
50/500
51/500
52/500
53/500
54/500
55/500
56/500
57/500
58/500
59/500
60/500
61/500
62/500
63/500
64/500
65/500
66/500
67/500
68/500
69/500
70/500
71/500
72/500
73/500
74/500
75/500
76/500
77/500
78/500
79/500
80/500
81/500
82/500
83/500
84/500
85/500
86/500
87/500
88/500
89/500
90/500
91/500
92/500
93/500
94/500
95/500
96/500
97/500
98/500
99/500
100/500
101/500
102/500
103/500
104/500
105/500
106/500
107/500
108/500
109/500
110/500
111/500
112/500
113/500
114/500
115/500
116/500
117/500
118/500
119/500
120/500
121/500
122/500
123/500
124/500
125/500
126/500
127/500
128/500
129/500
130/500
131/500
132/500
133/500
134/500
135/500
136/500
137/500
138/500
139/

In [27]:
def show_fav_list(counter, topN = 10):
    fav_list = sorted(counter.items(), key=lambda x:-x[1]['cnt'])
    cols = 'Heroes|Match Played|Win Rate|KDA(K/D/A)|Avg. KDA|Avg. GPM|Avg. XPM|\n'\
           ':-|:-|:-|:-|:-|:-|:-|\n'
    fmat = '%s|%d|%.1f%%|**%.1f**(%.1f/%.1f/%.1f)|%.1f|%.1f|%.1f|\n'
    if (topN > len(counter)):
        topN = len(counter)
    md = ''
    md += '### Most Played Heroes\n\n'
    md += cols
    for idx in range(topN):
        md += fmat%(get_png_md(heroes[heroid2idx[fav_list[idx][0]]]['portrait_url']), \
                    fav_list[idx][1]['cnt'], \
                    fav_list[idx][1]['win']/(fav_list[idx][1]['cnt']*1.0)*100.0, \
                    (fav_list[idx][1]['kills']+fav_list[idx][1]['assists'])/(fav_list[idx][1]['deaths']+1.0), \
                    fav_list[idx][1]['kills']/(fav_list[idx][1]['cnt']*1.0), \
                    fav_list[idx][1]['deaths']/(fav_list[idx][1]['cnt']*1.0), \
                    fav_list[idx][1]['assists']/(fav_list[idx][1]['cnt']*1.0), \
                    fav_list[idx][1]['KDA']/(fav_list[idx][1]['cnt']*1.0), \
                    fav_list[idx][1]['GPM']/(fav_list[idx][1]['cnt']*1.0), \
                    fav_list[idx][1]['XPM']/(fav_list[idx][1]['cnt']*1.0))
    printmd(md)
show_fav_list(counter)

### Most Played Heroes

Heroes|Match Played|Win Rate|KDA(K/D/A)|Avg. KDA|Avg. GPM|Avg. XPM|
:-|:-|:-|:-|:-|:-|:-|
![](http://cdn.dota2.com/apps/dota2/images/heroes/juggernaut_sb.png?raw=true)|34|64.7%|**5.2**(10.6/4.5/12.6)|5.7|579.0|596.6|
![](http://cdn.dota2.com/apps/dota2/images/heroes/mirana_sb.png?raw=true)|26|65.4%|**5.0**(8.9/4.7/14.8)|5.8|485.2|505.8|
![](http://cdn.dota2.com/apps/dota2/images/heroes/pudge_sb.png?raw=true)|22|68.2%|**3.0**(6.0/7.5/16.4)|2.9|395.7|476.2|
![](http://cdn.dota2.com/apps/dota2/images/heroes/nevermore_sb.png?raw=true)|21|81.0%|**4.8**(9.7/4.1/10.5)|7.0|595.9|600.0|
![](http://cdn.dota2.com/apps/dota2/images/heroes/rubick_sb.png?raw=true)|18|38.9%|**2.4**(2.9/6.6/12.9)|3.1|326.4|331.8|
![](http://cdn.dota2.com/apps/dota2/images/heroes/luna_sb.png?raw=true)|17|58.8%|**3.7**(7.3/5.5/13.4)|4.0|589.1|568.4|
![](http://cdn.dota2.com/apps/dota2/images/heroes/legion_commander_sb.png?raw=true)|16|68.8%|**3.4**(7.6/5.4/10.8)|3.5|458.2|499.0|
![](http://cdn.dota2.com/apps/dota2/images/heroes/life_stealer_sb.png?raw=true)|14|64.3%|**4.9**(10.9/4.8/13.1)|5.1|549.4|572.6|
![](http://cdn.dota2.com/apps/dota2/images/heroes/ursa_sb.png?raw=true)|14|50.0%|**3.9**(13.6/6.9/13.7)|5.6|518.0|554.2|
![](http://cdn.dota2.com/apps/dota2/images/heroes/slark_sb.png?raw=true)|14|57.1%|**4.3**(9.4/4.6/10.9)|4.5|502.0|542.4|


## Analyze a player: Radar chart

## Summary and references

1.

2.